<div style="text-align: right"><i>Peter Norvig, 2022</i></div> 

# The Babylonian Number System

Our number system has ten digit symbols, 0 to 9, which we concatenate to form larger integers with the understanding that we are using base 10, which means that each digit is worth ten times as much as the one to the right of it. For example:

$$409 = 4\times10^2 + 0\times10^1 + 9\times10^0 = 400 + 0 + 9 = 409$$


The ancient [**Babylonian cuneiform number system**](https://en.wikipedia.org/wiki/Babylonian_cuneiform_numerals) (circa 2000 BCE) had a number system that was different in three ways: 
1. They had no symbol for zero. 
2. It was base 60, not base 10. 
3. Instead of having 60 different symbols for the digits, they had just two symbols, for 1 and 10, and they used a tally system where you just put together as many 10s and 1s as you need (in any order) to make a "digit" from 1 to 59. 

Here are some examples:

![](https://xjwfriends.files.wordpress.com/2018/02/babyloniannumbers.jpg)

We will use the Unicode cuneiform characters `𒐕` to represent 1, and  `𒌋`  to represent 10.  If you have trouble finding the right Unicode characters you can use `!` for 1 and `<` for 10.

Thus,  59 can be represented as `𒌋𒌋𒌋𒌋𒌋𒐕𒐕𒐕𒐕𒐕𒐕𒐕𒐕𒐕` or `𒐕𒐕𒐕𒌋𒌋𒌋𒐕𒐕𒐕𒐕𒐕𒐕𒌋𒌋` or `<<<!!!!!<<!!!!`, for example.

Numbers larger than 59 use base 60 notation, with a single space separating base-60  "digits". So we have: 

`𒐕 𒌋𒐕𒐕𒐕𒐕𒐕` = `[1, 15]` = $1\times60^1 + 15\times 60^0 = 60 + 15 = 75$

and

`𒌋 𒐕𒐕𒐕 𒌋𒐕`  = `[10, 3, 11]` = $10\times60^2 + 3\times60^1 + 11\times60^0 = 36000 + 180 + 11 = 36191$

Without a zero, how did the Babylonians represent a number that had zero in, say, the 60s place? Their answer was to have two consecutive spaces with nothing in between. This makes perfect sense, but it is extremely error-prone to visually distinguish one space from two. Fortunately it is easier on the computer than on a clay tablet.

Here's a function to convert a Babylonian number string into an integer, with an auxiliary function to give the list of digits:

In [1]:
from typing import List

value = {'𒐕': 1, '𒌋': 10, '!': 1, '<': 10}
                     
def babyl_to_int(num: str) -> int:
    "Convert a base 60 Babylonian number string into an integer."
    return sum(d * 60 ** i for i, d in enumerate(reversed(digits(num))))

def digits(num: str) -> List[int]:
    "The list of integer digits for a Babylonian number"
    return [sum(value[c] for c in group) for group in num.split(' ')]

An example:

In [2]:
babyl_to_int('𒌋 𒐕𒐕𒐕 𒌋𒐕')

36191

In [3]:
digits('𒌋 𒐕𒐕𒐕 𒌋𒐕')

[10, 3, 11]

Now a function to go in the other direction:

In [4]:
def int_to_babyl(n: int) -> str:
    """Convert an integer into a base 60 Babylonian number string."""
    return ' '.join((d // 10) * '𒌋' + (d % 10) * '𒐕' 
                    for d in to_base(n, 60))
    
def to_base(n: int, base=60) -> List[int]:
    """The list of integer digits for `n` in base `base`."""
    return [n] if n < base else [*to_base(n // base, base), n % base]

In [5]:
int_to_babyl(36191)

'𒌋 𒐕𒐕𒐕 𒌋𒐕'

In [6]:
to_base(36191, 60)

[10, 3, 11]

Here is a more comprehensive test suite:

In [7]:
tests = [
    (     0,          [0], "", "  "),
    (     1,          [1], "𒐕", "!"),
    (    11,         [11], "𒌋𒐕", "<!"),
    (    59,         [59], "𒌋𒌋𒌋𒌋𒌋𒐕𒐕𒐕𒐕𒐕𒐕𒐕𒐕𒐕", "𒐕𒐕𒐕𒌋𒌋𒌋𒐕𒐕𒐕𒐕𒐕𒐕𒌋𒌋", "<<<!!!!!<<!!!!"),
    (    60,       [1, 0], "𒐕 "),
    (    61,       [1, 1], "𒐕 𒐕", "! !"),
    (  3601,    [1, 0, 1], "𒐕  𒐕"),
    (  4302,  [1, 11, 42], "𒐕 𒌋𒐕 𒌋𒌋𒌋𒌋𒐕𒐕", "𒐕 𒐕𒌋 𒌋𒌋𒐕𒐕𒌋𒌋", "! !< <<!!<<"),
    ( 36191,  [10, 3, 11], "𒌋 𒐕𒐕𒐕 𒌋𒐕"),
    (201600,   [56, 0, 0], "𒌋𒌋𒌋𒌋𒌋𒐕𒐕𒐕𒐕𒐕𒐕  "),
    (223384, [1, 2, 3, 4], "𒐕 𒐕𒐕 𒐕𒐕𒐕 𒐕𒐕𒐕𒐕")]

def test(tests=tests) -> bool:
    """Each test is an integer `n`, the digits in base 60, and 
    one or more ways of representing `n` as a Babylonian number strings."""
    for (n, base60_digits, *nums) in tests:
        assert int_to_babyl(n) in nums, f'int_to_babyl({n})'
        assert digits(nums[0]) == to_base(n, 60) == base60_digits, f'digits({nums[0]!r})'
        for num in nums:
            assert babyl_to_int(num) == n, f'babyl_to_int({num!r})'
    return True
    
test()

True